In [1]:
import os
import sys

import time
import copy
import math
import random
import numpy as np

from scipy.stats import mannwhitneyu

In [2]:
# 统计计算了多少次欧式距离（需要满足少于或等于20000*N次）
num_of_cal_pairs = 0


# 计算两个点之间的欧式距离
def euclidean_distance(point1, point2):
    global num_of_cal_pairs
    num_of_cal_pairs += 1
    return math.sqrt((point1[0]-point2[0])**2 + (point1[1]-point2[1])**2)


# 种群中的一个个体
# chromosome 为二维数组，每个旅行商的路径为一纬数组，不包括起点和终点
class Individual:

    def __init__(self):
        self.chromosome = None
        self.distance = None
        self.num_of_nodes = None
        self.num_of_salesman = None
    
    # 随机初始化，每个城市随机赋给一个旅行商的随机位置
    def random_init(self, num_of_nodes, num_of_salesman):
        self.num_of_nodes = num_of_nodes
        self.num_of_salesman = num_of_salesman
        self.chromosome = [[] for _ in range(num_of_salesman)]
        for node in range(1, num_of_nodes):
            sales = random.randint(0, num_of_salesman-1)
            self.chromosome[sales].insert(random.randint(0, len(self.chromosome[sales])), node)
    
    # 计算对应 chromosome 的总距离
    def cal_distance(self, coordinate_of_nodes):
        total_distance = 0.0
        for salesman in self.chromosome:
            distance = 0.0
            if len(salesman) != 0:
                distance += euclidean_distance(coordinate_of_nodes[0], 
                                               coordinate_of_nodes[salesman[0]])
                distance += euclidean_distance(coordinate_of_nodes[0], 
                                               coordinate_of_nodes[salesman[-1]])
            for ind in range(1, len(salesman)):
                distance += euclidean_distance(coordinate_of_nodes[salesman[ind-1]], 
                                               coordinate_of_nodes[salesman[ind]])
            total_distance += distance
        self.distance = total_distance
    
    # 对 chromosome 按照一定的概率变异
    # 一定的概率随机的两个城市互换（包括不同旅行商的位置）
    # 一定的概率随机将一个城市插入任意位置
    def mutate(self, probability):
        if random.uniform(0, 1) < probability:
            self.distance = None

            node1 = random.randint(1, self.num_of_nodes-1)
            node2 = random.randint(1, self.num_of_nodes-1)
            if node1 != node2:
                for i in range(len(self.chromosome)):
                    for j in range(len(self.chromosome[i])):
                        if self.chromosome[i][j] == node1:
                            self.chromosome[i][j] = node2
                        elif self.chromosome[i][j] == node2:
                            self.chromosome[i][j] = node1
        if random.uniform(0, 1) < probability:
            self.distance = None

            node = random.randint(1, self.num_of_nodes-1)
            sales = random.randint(0, self.num_of_salesman-1)
            if node not in self.chromosome[sales]:
                for i in range(len(self.chromosome)):
                    for j in range(len(self.chromosome[i])):
                        if self.chromosome[i][j] == node:
                            del self.chromosome[i][j]
                            break
                self.chromosome[sales].insert(random.randint(0, len(self.chromosome[sales])), node)
            

# 一整个演化的种群
# individuals 保存种群，offsprings 保存选择交叉后获得的子代
# survivals，把父代和子代放在一起，保存较好的个体
class Population:

    def __init__(self):
        self.individuals = []
        self.offsprings = []
        self.num_of_nodes = None
        self.num_of_salesman = None
        self.coordinate_of_nodes = None
    
    # 初始化种群
    def initialization(self, pop_size, num_of_nodes, num_of_salesman, coordinate_of_nodes):
        self.num_of_nodes = num_of_nodes
        self.num_of_salesman = num_of_salesman
        self.coordinate_of_nodes = coordinate_of_nodes
        self.individuals = []
        for _ in range(pop_size):
            individual = Individual()
            individual.random_init(num_of_nodes, num_of_salesman)
            self.individuals.append(individual)
    
    # 对所有 distance 为 None 的个体，计算距离
    def evaluation(self):
        for ind in self.individuals:
            if ind.distance is None:
                ind.cal_distance(self.coordinate_of_nodes)
        for off in self.offsprings:
            if off.distance is None:
                off.cal_distance(self.coordinate_of_nodes)
    
    # 选择，随机排列父代，两次随机排列对应的位置的，进行tournament selection，0.8的概率选好的，0.2的概率选坏，相同则各0.5
    def selection(self, probabilty=0.8):
        selection = []
        shuffle_1 = [i for i in range(len(self.individuals))]
        random.shuffle(shuffle_1)
        shuffle_2 = [i for i in range(len(self.individuals))]
        random.shuffle(shuffle_2)
        for i in range(len(self.individuals)):
            if shuffle_1[i] < shuffle_2[i]:
                selection.append(shuffle_1[i] if random.uniform(0, 1) < probabilty else shuffle_2[i])
            elif shuffle_1[i] > shuffle_2[i]:
                selection.append(shuffle_2[i] if random.uniform(0, 1) < probabilty else shuffle_1[i])
            else:
                selection.append(shuffle_1[i] if random.uniform(0, 1) < 0.5 else shuffle_2[i])
        return selection
    
    # 进行选择和交叉操作，获得子代
    def crossover(self, probabilty=0.5):
        # 进行两次选择，得到一组父代
        selection_1 = self.selection()
        selection_2 = self.selection()
        
        # 不进行交叉操作，对于每一对父代，0.5的概率，一对中较好的变成变成子代
        for i in range(len(selection_1)):
            if random.uniform(0, 1) > probabilty:
                best_one = self.individuals[selection_1[i]] if selection_1[i] < selection_2[i] else self.individuals[selection_2[i]]
                self.offsprings.append(copy.deepcopy(best_one))

    # 按照一定的概率对子代进行变异操作
    def mutation(self, probabilty=0.2):
        for off in self.offsprings:
            off.mutate(probabilty)
    
    # 子代和父代的个体放在一起，保留 distance 花销较小的个体
    def survival(self):
        darwin_space = []
        for ind in self.individuals:
            darwin_space.append(ind)
        for off in self.offsprings:
            darwin_space.append(off)

        darwin_space = sorted(darwin_space, key=lambda x:x.distance)
    
        self.individuals = darwin_space[:len(self.individuals)]
        self.offsprings = []

In [3]:
class JingranGA:
    
    def __init__(self):
        self.dataset_path = None
        self.num_of_nodes = None
        self.coordinate_of_nodes = None
        self.max_num_of_cal_pairs = None 
        
        self.generation = None
        self.best_distance_history = None
        self.best_route_history = None
        self.num_of_cal_pairs_history = None
        
        self.finished_time = None
    
    
    def fit(self, dataset_path):
        start_time = time.time()
        
        # 初始化 num_of_cal_pairs，读取数据集
        global num_of_cal_pairs
        num_of_cal_pairs = 0
        self.dataset_path = dataset_path
        self.num_of_nodes, self.coordinate_of_nodes = read_dataset(dataset_path)
        self.max_num_of_cal_pairs = self.num_of_nodes * 20000
        print('dataset loaded. the path of dataset {}'.format(dataset_path))
        print('num_of_cal_pairs should not larger than {}'.format(self.max_num_of_cal_pairs))
        
        self.best_distance_history = []
        self.best_route_history = []
        self.num_of_cal_pairs_history = []
        
        # 初始化种群
        population = Population()
        population.initialization(pop_size=100, 
                                  num_of_nodes=self.num_of_nodes, 
                                  num_of_salesman=5, 
                                  coordinate_of_nodes=self.coordinate_of_nodes)
        print('population init')
        
        
        # 进行选择交叉变异，直到达到停止条件
        self.generation = 0
        while True:
            # 判断是否超过停止条件，超过则停止，没超过进行survival操作
            population.evaluation()
            if num_of_cal_pairs > self.max_num_of_cal_pairs:
                break
            population.survival()
            self.generation += 1
            self.best_distance_history.append(population.individuals[0].distance)
            self.best_route_history.append(population.individuals[0].chromosome)
            self.num_of_cal_pairs_history.append(num_of_cal_pairs)
            
            last_num_of_cal_pairs = num_of_cal_pairs
            
            population.crossover()
            population.mutation()
        
        self.finished_time = time.time() - start_time
        print('finished, time {} seconds'.format(self.finished_time))
    
    def print_best(self):
        print('generation {}, num_of_cal_pairs {}, best distance {}\nbest route: {}'.format(self.generation, 
                                                                                            self.num_of_cal_pairs_history[-1], 
                                                                                            self.best_distance_history[-1], 
                                                                                            self.best_route_history[-1]))
        
    def save_history_to_file(self, filename, samplename):
        with open(filename, 'a+') as f:
            f.write('------\n')
            f.write('{}, {}, {}, {}\n'.format(samplename, 
                                              self.dataset_path, 
                                              self.max_num_of_cal_pairs, 
                                              self.finished_time))
            for i in range(self.generation):
                f.write('{}, {}, {}, {}\n'.format(i+1, 
                                              self.best_distance_history[i], 
                                              self.num_of_cal_pairs_history[i], 
                                              self.best_route_history[i]))
            f.write('\n\n')

In [4]:
# 读取数据集到 numpy ndarray 数组
def read_dataset(filepath):
    with open(filepath) as f:
        lines = [line.rstrip('\n') for line in f.readlines()]
        
    num_of_nodes = int(lines[0])
    coordinate_of_nodes = np.zeros((num_of_nodes, 2))
    for line in lines[1:]:
        line = line.split(' ')
        if len(line) != 3:
            break
        coordinate_of_nodes[int(line[0])-1, 0] = float(line[1])
        coordinate_of_nodes[int(line[0])-1, 1] = float(line[2])
    
    return num_of_nodes, coordinate_of_nodes


def extractFinalDistFromLog(log_path):
    if not os.path.exists(log_path):
        sys.stderr.write('[extractFinalDistFromLog] %s does not exist!\n' % log_path)
        exit(-1000)

    with open(log_path) as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            if line.startswith('> Final distance'):
                return float(line.split(' ')[-1])


def extractFinalDistListFromLogDir(log_dir):
    if not os.path.exists(log_dir):
        sys.stderr.write('[extractFinalDistListFromLogDir] %s does not exist!\n' % log_dir)
        exit(-1001)

    res = []
    for file in os.listdir(log_dir):
        res.append(extractFinalDistFromLog(os.path.join(log_dir, file)))
    return res

In [5]:
jingran_socool = JingranGA()

In [6]:
start_time = time.time()

dataset = 'mtsp51'
file_path = './TSP/{}.txt'.format(dataset)
final_result_list = []
time_list = []
for i in range(1, 31):
    jingran_socool.fit(file_path)
    jingran_socool.print_best()
    jingran_socool.save_history_to_file('{}_output.log'.format(dataset), '{}_{}'.format(dataset, i))
    final_result_list.append(jingran_socool.best_distance_history[-1])
    time_list.append(jingran_socool.finished_time)

print('distance list: {}'.format(final_result_list))
print('time list: {}'.format(time_list))
print('avg route: {}, avg time: {} seconds'.format(sum(final_result_list)/30, sum(time_list)/30))
print('total time: {} seconds'.format(time.time()-start_time))

dataset loaded. the path of dataset ./TSP/mtsp51.txt
num_of_cal_pairs should not larger than 1020000
population init
finished, time 3.4944419860839844 seconds
generation 1012, num_of_cal_pairs 1019535, best distance 557.3619026388351
best route: [[5, 13, 24, 12, 40, 39, 18, 41, 3, 17, 46, 45, 50, 26], [31, 4, 48, 37, 10], [47, 23, 42, 22, 6, 25, 7, 30, 27, 2, 35, 34, 19], [11, 16, 36, 43, 14, 44, 32, 38, 9, 29, 33, 49, 8], [1, 15, 20, 28, 21]]
dataset loaded. the path of dataset ./TSP/mtsp51.txt
num_of_cal_pairs should not larger than 1020000
population init
finished, time 3.3978826999664307 seconds
generation 1021, num_of_cal_pairs 1019755, best distance 575.2664150305491
best route: [[26, 50, 45], [1, 28, 19, 34, 35, 2, 30, 27, 7, 25, 6, 22, 42, 23, 13, 17, 3, 46], [21, 15, 49, 20, 33, 29, 8], [10, 37, 48, 9, 38, 32, 44, 14, 4, 31], [11, 16, 36, 43, 41, 18, 39, 40, 12, 24, 5, 47]]
dataset loaded. the path of dataset ./TSP/mtsp51.txt
num_of_cal_pairs should not larger than 1020000
pop

finished, time 3.468721389770508 seconds
generation 1032, num_of_cal_pairs 1019645, best distance 549.747918863603
best route: [[37, 48, 9, 38, 32, 44, 14, 36, 4, 31], [26, 50, 5, 13, 24, 17, 3, 12, 40, 39, 18, 41, 43, 16, 46, 11, 45], [47, 22, 23, 42, 6, 25, 7, 30, 27, 21], [2, 35, 34, 19, 28, 1], [15, 49, 20, 33, 29, 8, 10]]
dataset loaded. the path of dataset ./TSP/mtsp51.txt
num_of_cal_pairs should not larger than 1020000
population init
finished, time 3.477090835571289 seconds
generation 1028, num_of_cal_pairs 1018930, best distance 589.7005777557796
best route: [[10, 37, 48, 8, 29, 33, 20, 49, 15, 28, 1], [7, 25, 6, 42, 23, 24, 13, 5, 26], [31, 11, 36, 9, 38, 32, 44, 14, 4, 2, 35, 34, 19], [30, 27, 21], [45, 50, 46, 3, 16, 43, 41, 18, 39, 40, 12, 17, 22, 47]]
dataset loaded. the path of dataset ./TSP/mtsp51.txt
num_of_cal_pairs should not larger than 1020000
population init
finished, time 3.421257972717285 seconds
generation 1037, num_of_cal_pairs 1019590, best distance 570.89466

In [7]:
logDir = './res/baseline/log/mtsp51_baseline_30_5_1000_100_10_20211216_16_12_09'
baseline_result_list = extractFinalDistListFromLogDir(logDir)


print(mannwhitneyu(final_result_list, baseline_result_list, alternative='two-sided'))
print('avg of JingranGA: {}'.format(sum(final_result_list)/30))
print('avg of baseline: {}'.format(sum(baseline_result_list)/30))

MannwhitneyuResult(statistic=0.0, pvalue=3.019859359162157e-11)
avg of JingranGA: 580.1120456126616
avg of baseline: 918.3617979333334


In [8]:
start_time = time.time()

dataset = 'mtsp100'
file_path = './TSP/{}.txt'.format(dataset)
final_result_list = []
time_list = []
for i in range(1, 31):
    jingran_socool.fit(file_path)
    jingran_socool.print_best()
    jingran_socool.save_history_to_file('{}_output.log'.format(dataset), '{}_{}'.format(dataset, i))
    final_result_list.append(jingran_socool.best_distance_history[-1])
    time_list.append(jingran_socool.finished_time)

print('distance list: {}'.format(final_result_list))
print('time list: {}'.format(time_list))
print('avg route: {}, avg time: {} seconds'.format(sum(final_result_list)/30, sum(time_list)/30))
print('total time: {} seconds'.format(time.time()-start_time))

dataset loaded. the path of dataset ./TSP/mtsp100.txt
num_of_cal_pairs should not larger than 2000000
population init
finished, time 5.852729320526123 seconds
generation 1074, num_of_cal_pairs 1997840, best distance 50712.98029224838
best route: [[72, 3, 70, 38, 73, 60, 75, 13, 67, 81, 94, 55, 20, 64, 30, 77, 79, 65, 37], [87, 84, 17, 32, 44, 95, 85, 58, 2, 52, 50, 29, 99, 36, 34, 61, 98, 56, 71, 23, 78, 12], [10, 88, 48, 69, 21, 41, 22, 51, 7, 6, 74, 35, 66, 28, 57, 16, 93, 31, 90, 39, 82, 53, 83], [26, 40, 92, 43, 59, 18, 1, 47, 91, 25, 86, 54, 68, 80, 33, 89, 97, 96], [14, 62, 76, 42, 4, 8, 9, 27, 11, 46, 24, 19, 5, 63, 15, 45, 49]]
dataset loaded. the path of dataset ./TSP/mtsp100.txt
num_of_cal_pairs should not larger than 2000000
population init
finished, time 5.833078861236572 seconds
generation 1060, num_of_cal_pairs 1997944, best distance 49598.61105334595
best route: [[23, 38, 70, 93, 89, 88, 22, 73, 60, 6, 79, 77, 64, 85, 62, 40], [49, 35, 74, 66, 75, 13, 2, 4, 18, 50, 1, 29

finished, time 5.722581148147583 seconds
generation 1054, num_of_cal_pairs 1997424, best distance 47367.084755248274
best route: [[23, 83, 53, 38, 72, 40, 93, 31, 96, 97, 21, 28, 57, 35, 74, 66, 73, 60, 77, 5, 84], [33, 65, 45, 75, 79, 64, 9, 8, 25, 86, 91, 4, 52, 3, 42, 76, 90, 26], [10, 22, 37, 70, 43, 59, 61, 36, 18, 1, 47, 54, 2, 39, 82, 95, 85, 68], [49, 14, 62, 63, 58, 17, 24, 19, 20, 27, 11, 44, 32, 67, 13, 46, 55, 94, 81, 98, 30, 56, 71, 6, 41, 88, 89, 69, 48, 16], [78, 92, 50, 99, 29, 34, 15, 51, 7, 80, 87, 12]]
dataset loaded. the path of dataset ./TSP/mtsp100.txt
num_of_cal_pairs should not larger than 2000000
population init
finished, time 5.733498573303223 seconds
generation 1063, num_of_cal_pairs 1998360, best distance 51095.842200992396
best route: [[26, 85, 11, 27, 20, 9, 54, 25, 86, 47, 1, 99, 29, 34, 52, 78, 12], [68, 62, 15, 63, 84, 95, 64, 71, 56, 74, 66, 35, 28, 57, 10, 16, 93, 31], [49, 40, 72, 38, 70, 59, 4, 2, 75, 60, 65, 51, 45, 42, 3, 43, 92, 23], [83, 14, 58,

finished, time 5.737111568450928 seconds
generation 1054, num_of_cal_pairs 1999088, best distance 50711.80125629553
best route: [[12, 78, 70, 92, 34, 18, 36, 2, 58, 6, 74, 66, 73, 60, 56, 30, 77, 22, 80, 33], [87, 23, 83, 72, 3, 52, 59, 43, 39, 5, 15, 62], [10, 89, 16, 96, 48, 97, 69, 41, 7, 45, 14, 90, 53, 42, 63, 82, 19, 8, 24, 17, 84, 76], [31, 93, 88, 21, 28, 57, 35, 71, 98, 32, 67, 13, 61, 50, 99, 29, 1, 47, 91, 54, 86, 25, 4, 40, 26], [79, 75, 85, 81, 46, 11, 44, 64, 51, 65, 37, 38, 9, 20, 27, 55, 94, 95, 68, 49]]
dataset loaded. the path of dataset ./TSP/mtsp100.txt
num_of_cal_pairs should not larger than 2000000
population init
finished, time 5.686746835708618 seconds
generation 1055, num_of_cal_pairs 1998464, best distance 52672.187425113574
best route: [[80, 7, 73, 60, 75, 95, 85, 91, 86, 25, 54, 8, 46, 27, 11, 20, 24, 84, 72, 12], [49, 57, 28, 71, 56, 30, 63, 59, 43, 52, 82, 39, 83, 26, 23, 87, 78], [31, 88, 48, 69, 65, 98, 44, 32, 67, 13, 81, 18, 50, 76, 51, 37, 10, 89, 16,

In [9]:
logDir = './res/baseline/log/mtsp100_baseline_30_5_1000_100_10_20211216_16_28_21'
baseline_result_list = extractFinalDistListFromLogDir(logDir)


print(mannwhitneyu(final_result_list, baseline_result_list, alternative='two-sided'))
print('avg of JingranGA: {}'.format(sum(final_result_list)/30))
print('avg of baseline: {}'.format(sum(baseline_result_list)/30))

MannwhitneyuResult(statistic=0.0, pvalue=3.019859359162157e-11)
avg of JingranGA: 49443.215262466234
avg of baseline: 83753.54461313334


In [10]:
start_time = time.time()

dataset = 'mtsp150'
file_path = './TSP/{}.txt'.format(dataset)
final_result_list = []
time_list = []
for i in range(1, 31):
    jingran_socool.fit(file_path)
    jingran_socool.print_best()
    jingran_socool.save_history_to_file('{}_output.log'.format(dataset), '{}_{}'.format(dataset, i))
    final_result_list.append(jingran_socool.best_distance_history[-1])
    time_list.append(jingran_socool.finished_time)

print('distance list: {}'.format(final_result_list))
print('time list: {}'.format(time_list))
print('avg route: {}, avg time: {} seconds'.format(sum(final_result_list)/30, sum(time_list)/30))
print('total time: {} seconds'.format(time.time()-start_time))

dataset loaded. the path of dataset ./TSP/mtsp150.txt
num_of_cal_pairs should not larger than 3000000
population init
finished, time 8.196278095245361 seconds
generation 1074, num_of_cal_pairs 2999920, best distance 86206.20687697723
best route: [[132, 88, 147, 141, 131, 46, 85, 59, 6, 42, 127, 13, 70, 16, 145, 51, 40, 45, 106, 38, 68, 2, 118, 117, 128, 52, 142, 41, 78, 33, 148, 134], [100, 55, 66, 21, 102, 146, 43, 1, 67, 49, 29, 95, 77, 105, 61, 26, 104, 56, 47, 111], [50, 144, 116, 19, 22, 149, 11, 114, 82, 99, 120, 80, 4, 84, 34, 90, 10, 62, 74, 96, 138, 7, 79, 48, 89, 20, 73, 58, 87, 103, 76, 109, 60], [97, 44, 8, 119, 122, 124, 135, 28, 86, 64, 75, 12, 125, 81, 63, 92, 137, 30, 18, 65, 69, 93, 136, 37, 35, 25, 101, 5, 130, 139, 54], [57, 27, 129, 112, 23, 133, 123, 3, 121, 39, 53, 143, 91, 71, 126, 140, 14, 31, 9, 15, 17, 110, 98, 83, 108, 113, 32, 36, 94, 115, 72, 24, 107]]
dataset loaded. the path of dataset ./TSP/mtsp150.txt
num_of_cal_pairs should not larger than 3000000
popu

finished, time 8.067874670028687 seconds
generation 1079, num_of_cal_pairs 2997918, best distance 92191.14734586538
best route: [[3, 96, 24, 139, 134, 106, 29, 36, 81, 1, 39, 92, 137, 88, 143, 53, 144, 60, 130, 78, 129, 27, 141, 79, 7, 118, 117, 52, 18, 22, 149], [41, 91, 5, 71, 57, 2, 84, 13, 124, 105, 80, 102, 66, 28, 135, 104, 56, 95, 38, 146, 21, 100, 64, 55, 49, 115, 63, 107], [6, 8, 16, 51, 61, 86, 120, 77, 75, 32, 4, 67, 136, 15, 133, 142, 30, 50, 19, 34, 148, 114, 122, 11, 119, 116, 72, 68], [89, 23, 9, 112, 31, 46, 47, 99, 45, 76, 73, 10, 108, 59, 111, 54, 94, 12, 125, 43, 147, 132, 121, 138, 103, 87, 25, 62, 140, 14, 109, 127, 42, 145, 40, 82, 70, 26, 33], [85, 90, 113, 44, 97, 126, 20, 58, 83, 101, 35, 37, 98, 17, 110, 93, 69, 123, 65, 128, 74, 48, 131]]
dataset loaded. the path of dataset ./TSP/mtsp150.txt
num_of_cal_pairs should not larger than 3000000
population init
finished, time 8.104673385620117 seconds
generation 1071, num_of_cal_pairs 2998380, best distance 91271.99

finished, time 8.156855583190918 seconds
generation 1085, num_of_cal_pairs 2998996, best distance 96255.89539248704
best route: [[76, 149, 22, 6, 19, 34, 90, 10, 108, 59, 8, 102, 75, 32, 47, 26, 61, 51, 70, 42, 85, 71, 20, 140, 62, 113, 145, 86, 21, 66, 100, 64, 94, 7, 138, 142, 30], [144, 84, 54, 67, 68, 3, 117, 96, 118, 65, 17, 110, 101, 98, 37, 74, 107, 39, 147, 132, 41, 129, 57, 116, 134, 80, 4, 95, 29], [50, 60, 139, 13, 99, 122, 97, 14, 58, 103, 130, 24, 33, 119, 11, 114, 106, 111, 104, 16, 127, 82, 77, 55, 12, 81, 115], [109, 148, 45, 40, 56, 28, 135, 38, 72, 63, 137, 88, 121, 91, 48, 89, 5, 112, 15, 35, 126, 131, 141, 2, 92, 79, 18, 133, 136], [44, 73, 87, 25, 83, 9, 52, 123, 128, 69, 93, 23, 31, 46, 78, 27, 49, 146, 43, 143, 53, 1, 125, 36, 120, 124, 105]]
dataset loaded. the path of dataset ./TSP/mtsp150.txt
num_of_cal_pairs should not larger than 3000000
population init
finished, time 7.998209476470947 seconds
generation 1067, num_of_cal_pairs 2999304, best distance 98294.65

In [11]:
logDir = './res/baseline/log/mtsp150_baseline_30_5_1000_100_10_20211216_16_50_57'
baseline_result_list = extractFinalDistListFromLogDir(logDir)


print(mannwhitneyu(final_result_list, baseline_result_list, alternative='two-sided'))
print('avg of JingranGA: {}'.format(sum(final_result_list)/30))
print('avg of baseline: {}'.format(sum(baseline_result_list)/30))

MannwhitneyuResult(statistic=0.0, pvalue=3.019859359162157e-11)
avg of JingranGA: 94633.47115046621
avg of baseline: 143786.4989089333


In [12]:
start_time = time.time()

dataset = 'pr76'
file_path = './TSP/{}.txt'.format(dataset)
final_result_list = []
time_list = []
for i in range(1, 31):
    jingran_socool.fit(file_path)
    jingran_socool.print_best()
    jingran_socool.save_history_to_file('{}_output.log'.format(dataset), '{}_{}'.format(dataset, i))
    final_result_list.append(jingran_socool.best_distance_history[-1])
    time_list.append(jingran_socool.finished_time)

print('distance list: {}'.format(final_result_list))
print('time list: {}'.format(time_list))
print('avg route: {}, avg time: {} seconds'.format(sum(final_result_list)/30, sum(time_list)/30))
print('total time: {} seconds'.format(time.time()-start_time))

dataset loaded. the path of dataset ./TSP/pr76.txt
num_of_cal_pairs should not larger than 1520000
population init
finished, time 4.71063494682312 seconds
generation 1048, num_of_cal_pairs 1519680, best distance 195000.72874897037
best route: [[1, 7, 73, 12, 13, 10, 34, 35, 16, 15, 14, 11, 6], [23, 43, 47, 52, 40, 59, 58, 57, 56, 64, 65, 49, 48, 51, 50, 55, 54, 53, 41, 42, 25, 26, 20], [5, 8, 9, 17, 36, 37, 38, 39, 31, 3, 2], [22, 45, 46, 67, 66, 69, 68, 44, 24, 21, 75, 74], [4, 19, 18, 28, 27, 63, 62, 70, 71, 72, 61, 60, 33, 32, 29, 30]]
dataset loaded. the path of dataset ./TSP/pr76.txt
num_of_cal_pairs should not larger than 1520000
population init
finished, time 4.6175291538238525 seconds
generation 1059, num_of_cal_pairs 1518720, best distance 198212.39670837554
best route: [[75, 74, 7, 11, 15, 14, 38, 39, 54, 55, 56, 61, 57, 50, 49, 48, 47, 43, 24], [1, 2, 4, 28, 42, 41, 53, 52, 51, 65, 64, 66, 69, 67, 68, 46, 23, 21], [5, 8, 9, 10, 16, 17, 36, 29, 30, 18, 19, 3], [22, 45, 44, 40

finished, time 4.614206075668335 seconds
generation 1061, num_of_cal_pairs 1518320, best distance 210485.30122989428
best route: [[18, 29, 35, 16, 10, 14, 15, 17, 36, 4], [74, 75, 22, 41, 53, 55, 70, 71, 72, 56, 57, 58, 40, 39, 33, 19], [23, 43, 47, 48, 50, 65, 64, 63, 62, 61, 60, 59, 32, 31, 30], [24, 25, 28, 34, 37, 38, 54, 51, 67, 69, 66, 49, 52, 42, 27, 26, 20], [1, 73, 13, 12, 11, 9, 8, 7, 6, 5, 2, 3, 44, 46, 68, 45, 21]]
dataset loaded. the path of dataset ./TSP/pr76.txt
num_of_cal_pairs should not larger than 1520000
population init
finished, time 4.710670471191406 seconds
generation 1049, num_of_cal_pairs 1519200, best distance 208528.1382902188
best route: [[6, 7, 8, 10, 16, 36, 35, 38, 37, 17, 5], [53, 54, 56, 70, 71, 72, 63, 62, 57, 32, 28, 29], [26, 52, 51, 50, 64, 65, 55, 33, 34, 30, 18, 4, 2], [1, 73, 13, 12, 14, 15, 11, 9, 24, 23, 45, 44, 47, 41, 39, 40, 59, 58, 60, 61, 31, 20, 21, 22], [27, 42, 48, 49, 66, 69, 67, 68, 46, 43, 25, 19, 3, 74, 75]]
dataset loaded. the path

In [13]:
logDir = './res/baseline/log/pr76_baseline_30_5_1000_100_10_20211216_17_32_14'
baseline_result_list = extractFinalDistListFromLogDir(logDir)


print(mannwhitneyu(final_result_list, baseline_result_list, alternative='two-sided'))
print('avg of JingranGA: {}'.format(sum(final_result_list)/30))
print('avg of baseline: {}'.format(sum(baseline_result_list)/30))

MannwhitneyuResult(statistic=0.0, pvalue=3.019859359162157e-11)
avg of JingranGA: 204902.0892281582
avg of baseline: 338887.52780916664


In [14]:
start_time = time.time()

dataset = 'pr152'
file_path = './TSP/{}.txt'.format(dataset)
final_result_list = []
time_list = []
for i in range(1, 31):
    jingran_socool.fit(file_path)
    jingran_socool.print_best()
    jingran_socool.save_history_to_file('{}_output.log'.format(dataset), '{}_{}'.format(dataset, i))
    final_result_list.append(jingran_socool.best_distance_history[-1])
    time_list.append(jingran_socool.finished_time)

print('distance list: {}'.format(final_result_list))
print('time list: {}'.format(time_list))
print('avg route: {}, avg time: {} seconds'.format(sum(final_result_list)/30, sum(time_list)/30))
print('total time: {} seconds'.format(time.time()-start_time))

dataset loaded. the path of dataset ./TSP/pr152.txt
num_of_cal_pairs should not larger than 3040000
population init
finished, time 8.276954412460327 seconds
generation 1078, num_of_cal_pairs 3038568, best distance 337096.5932109207
best route: [[33, 14, 2, 3, 81, 144, 143, 118, 62, 40, 9, 8, 16, 10, 22, 26, 67, 43, 86, 136, 137, 134, 145, 151, 149, 124, 123, 125, 150, 98, 15], [36, 30, 4, 38, 84, 83, 42, 54, 95, 114, 92, 76, 47, 23, 5, 25, 24, 11, 61, 105, 104, 128, 147, 148, 129, 130, 131, 120, 112, 55, 65, 17, 18, 7, 39, 21, 13, 35], [51, 52, 70, 69, 53, 44, 88, 99, 113, 97, 115, 72, 49, 75, 73, 48, 50, 74, 71, 56, 64, 29, 28, 66, 141, 117, 140, 142, 132, 133, 119, 96], [68, 80, 111, 103, 85, 59, 60, 58, 82, 41, 19, 20, 6, 101, 87, 110, 106, 109, 108, 107, 63, 57, 79, 89], [34, 46, 77, 78, 91, 93, 126, 127, 122, 90, 45, 31, 1, 32, 94, 121, 146, 135, 138, 139, 116, 100, 102, 27, 12, 37]]
dataset loaded. the path of dataset ./TSP/pr152.txt
num_of_cal_pairs should not larger than 304000

finished, time 8.147881507873535 seconds
generation 1074, num_of_cal_pairs 3038412, best distance 292061.1909756143
best route: [[34, 50, 46, 49, 73, 72, 55, 64, 104, 105, 106, 84, 107, 62, 39, 7, 19, 16, 17, 8, 12, 48, 74, 91, 78, 65, 57, 108, 109, 85, 59, 41, 3], [35, 95, 125, 124, 151, 129, 144, 142, 116, 139, 141, 140, 70, 45, 52, 115, 93, 76, 94, 148, 146, 131, 117, 137, 138, 136, 145, 133, 119, 132, 135, 118, 134, 143, 97, 77], [36, 14, 51, 90, 114, 82, 63, 61, 60, 40, 83, 98, 96, 89, 79, 101, 81, 58, 6, 9, 20, 18, 10, 22, 38, 25, 26], [2, 13, 28, 30, 29, 31, 1, 15, 47, 75, 92, 130, 120, 128, 147, 121, 80, 43, 5, 21, 24, 23, 27, 37, 32], [33, 4, 11, 149, 127, 112, 100, 86, 103, 111, 110, 87, 102, 150, 126, 123, 122, 53, 68, 69, 113, 88, 99, 54, 67, 66, 56, 42, 44, 71]]
dataset loaded. the path of dataset ./TSP/pr152.txt
num_of_cal_pairs should not larger than 3040000
population init
finished, time 8.191620826721191 seconds
generation 1076, num_of_cal_pairs 3039504, best distance 

finished, time 8.240606546401978 seconds
generation 1081, num_of_cal_pairs 3039660, best distance 350265.9632606557
best route: [[96, 97, 112, 88, 64, 65, 42, 56, 78, 90, 77, 94, 144, 134, 109, 108, 107, 86, 66, 43, 69, 52, 71, 51, 45, 9, 39, 99, 113, 122, 127, 48, 47, 36, 15], [30, 29, 87, 100, 111, 110, 81, 2, 13, 27, 26, 3, 6, 20, 19, 18, 7, 17, 61, 60, 59, 21, 11, 103, 119, 132, 116, 138, 139, 84, 40, 32], [75, 44, 80, 101, 82, 57, 58, 63, 49, 74, 114, 140, 137, 117, 136, 135, 145, 50, 73, 72, 70, 53, 67, 55, 54, 41, 16, 8, 10, 5, 4, 1], [28, 12, 38, 22, 83, 85, 106, 102, 79, 95, 115, 91, 76, 46, 37, 24, 23, 25, 93, 92, 123, 126, 128, 147, 89, 35, 34], [33, 14, 31, 68, 149, 151, 125, 124, 150, 129, 121, 148, 146, 133, 118, 143, 130, 131, 120, 98, 141, 142, 105, 104, 62]]
dataset loaded. the path of dataset ./TSP/pr152.txt
num_of_cal_pairs should not larger than 3040000
population init
finished, time 8.212534189224243 seconds
generation 1071, num_of_cal_pairs 3039504, best distance 

finished, time 8.246069431304932 seconds
generation 1078, num_of_cal_pairs 3039036, best distance 339492.15033330146
best route: [[33, 52, 78, 89, 98, 69, 71, 70, 148, 129, 147, 146, 130, 122, 149, 123, 151, 103, 85, 83, 82, 63, 57, 25, 26, 28], [55, 62, 61, 106, 105, 104, 111, 110, 87, 119, 145, 134, 118, 133, 144, 143, 56, 112, 113, 88, 127, 125, 124, 94, 46], [51, 96, 97, 79, 43, 6, 39, 7, 9, 18, 22, 42, 66, 65, 80, 99, 132, 120, 131, 126, 150, 92, 93, 91, 72, 4, 2], [58, 59, 60, 17, 16, 8, 24, 29, 30, 31, 32, 34, 1, 3, 27, 12, 37, 45, 68, 53, 101, 102, 100, 50, 74, 49, 75, 47, 73, 76, 64, 86, 81, 54, 36, 15, 35], [140, 142, 136, 107, 40, 5, 19, 20, 10, 38, 135, 139, 141, 116, 138, 137, 117, 108, 109, 84, 41, 13, 14, 48, 114, 95, 90, 44, 77, 115, 128, 121, 67, 21, 23, 11]]
distance list: [337096.5932109207, 362568.4126652137, 349634.421057091, 338485.85942435055, 336487.8645607298, 312154.72706797055, 348518.4536928412, 344289.02796753583, 333084.25582583144, 292061.1909756143, 3072

In [15]:
logDir = './res/baseline/log/pr152_baseline_30_5_1000_100_10_20211216_19_13_47'
baseline_result_list = extractFinalDistListFromLogDir(logDir)


print(mannwhitneyu(final_result_list, baseline_result_list, alternative='two-sided'))
print('avg of JingranGA: {}'.format(sum(final_result_list)/30))
print('avg of baseline: {}'.format(sum(baseline_result_list)/30))

MannwhitneyuResult(statistic=0.0, pvalue=3.019859359162157e-11)
avg of JingranGA: 328849.77738046093
avg of baseline: 541097.5042658999


In [16]:
start_time = time.time()

dataset = 'pr226'
file_path = './TSP/{}.txt'.format(dataset)
final_result_list = []
time_list = []
for i in range(1, 31):
    jingran_socool.fit(file_path)
    jingran_socool.print_best()
    jingran_socool.save_history_to_file('{}_output.log'.format(dataset), '{}_{}'.format(dataset, i))
    final_result_list.append(jingran_socool.best_distance_history[-1])
    time_list.append(jingran_socool.finished_time)

print('distance list: {}'.format(final_result_list))
print('time list: {}'.format(time_list))
print('avg route: {}, avg time: {} seconds'.format(sum(final_result_list)/30, sum(time_list)/30))
print('total time: {} seconds'.format(time.time()-start_time))

dataset loaded. the path of dataset ./TSP/pr226.txt
num_of_cal_pairs should not larger than 4520000
population init
finished, time 11.749886751174927 seconds
generation 1083, num_of_cal_pairs 4515820, best distance 541857.0338764692
best route: [[5, 7, 21, 20, 18, 66, 16, 57, 58, 31, 23, 22, 51, 54, 56, 70, 4, 2, 216, 219, 215, 211, 176, 159, 160, 157, 162, 158, 156, 152, 150, 151, 113, 115, 103, 19, 49, 48, 92, 141, 140, 193, 199, 196, 198, 192, 132, 129], [14, 17, 30, 39, 154, 171, 178, 135, 134, 175, 170, 169, 166, 164, 179, 187, 191, 107, 71, 13, 10, 15, 47, 36, 24, 25, 106, 111, 117, 116, 168, 155, 40, 41, 32, 26, 45, 101, 145, 206, 214, 222, 221], [165, 167, 163, 188, 186, 189, 195, 202, 201, 210, 133, 42, 33, 28, 29, 95, 99, 100, 153, 180, 190, 205, 204, 203, 225, 224, 218, 123, 104, 110, 102, 6, 1, 128, 121, 118, 112], [55, 35, 44, 46, 27, 43, 34, 37, 38, 50, 52, 53, 60, 59, 73, 72, 67, 114, 105, 109, 131, 220, 209, 207, 200, 124, 81, 84, 82, 197, 194, 108, 77, 78, 91, 89, 97, 

finished, time 11.60287070274353 seconds
generation 1086, num_of_cal_pairs 4516970, best distance 623718.9989927055
best route: [[3, 6, 8, 69, 52, 23, 34, 39, 38, 40, 152, 142, 156, 167, 185, 190, 206, 205, 174, 164, 155, 154, 158, 157, 183, 182, 209, 213, 212, 210, 86, 95, 88, 54, 16, 72, 76, 74, 168, 161, 159, 163, 171, 81, 77, 78, 4, 18, 61, 62, 58, 55, 26, 27, 50], [116, 198, 197, 192, 175, 176, 108, 109, 21, 32, 146, 145, 217, 218, 129, 103, 104, 63, 20, 29, 37, 47, 35, 45, 153, 160, 162, 119, 125, 118, 117, 112, 15, 5], [67, 220, 214, 199, 186, 181, 187, 184, 12, 2, 1, 111, 120, 122, 207, 211, 203, 204, 202, 73, 75, 79, 123, 170, 178, 177, 189, 121, 127, 17, 13, 11, 90, 100, 99, 87, 105, 102, 85, 80, 219, 224, 216, 172, 165, 150, 151, 222, 225, 208, 195, 132, 134, 10], [131, 193, 194, 196, 24, 25, 28, 49, 7, 9, 14, 19, 98, 101, 93, 68, 83, 82, 84, 110, 115, 107, 53, 56, 66, 33, 36, 42, 91, 96, 94, 44, 48, 133, 135, 215, 223, 221, 41, 43, 71], [70, 65, 149, 166, 169, 173, 180, 179

finished, time 11.755046129226685 seconds
generation 1091, num_of_cal_pairs 4517430, best distance 616618.3599939132
best route: [[146, 142, 196, 200, 35, 33, 34, 32, 30, 25, 29, 45, 95, 137, 136, 129, 126, 53, 21, 48, 44, 38, 41, 56, 58, 51, 52, 214, 213, 205, 203, 201, 189, 181, 10, 14, 12, 18, 148, 163, 164, 107, 116, 81, 80, 79, 82, 118, 120, 125, 1], [4, 72, 83, 84, 85, 9, 5, 2, 115, 134, 141, 186, 170, 159, 157, 150, 97, 98, 94, 131, 123, 130, 174, 166, 168, 167, 162, 160, 161, 158, 152, 151, 153, 144, 99, 106, 105, 103, 73], [220, 211, 71, 8, 13, 64, 63, 54, 86, 91, 156, 154, 155, 39, 42, 46, 219, 210, 145, 40, 43, 36, 37, 104, 102, 112, 59, 61, 62, 60, 187, 191, 206, 202, 188, 173, 179, 19, 6, 3, 124, 119, 55, 57, 28, 31, 68, 67], [122, 121, 128, 108, 109, 78, 74, 77, 224, 225, 209, 208, 204, 180, 172, 171, 182, 184, 183, 176, 185, 194, 195, 199, 11, 7, 114, 218, 207, 198, 197, 178, 165, 149, 138, 143, 139, 140, 169, 175, 177, 76, 75, 20, 47, 27, 23, 26, 133, 132, 89, 88, 66, 7

finished, time 11.561165809631348 seconds
generation 1081, num_of_cal_pairs 4517890, best distance 553903.9390329985
best route: [[110, 116, 120, 131, 124, 213, 220, 211, 209, 206, 204, 139, 144, 136, 190, 217, 218, 214, 205, 201, 182, 184, 187, 188, 186, 134, 103, 112, 104, 65, 15, 7, 4, 70, 138, 86, 101, 212, 215, 210, 178, 171, 169, 166, 177, 185, 191, 199, 123, 121, 133, 174, 167, 170, 142], [17, 63, 94, 96, 59, 58, 55, 132, 225, 221, 129, 118, 107, 23, 37, 38, 39, 41, 98, 91, 143, 147, 164, 163, 196, 195, 200, 128, 127, 172, 173, 179, 145, 146, 148, 99, 97, 89, 48, 51, 50, 47, 30, 43, 152, 153, 93, 87, 69, 71], [111, 102, 72, 82, 74, 3, 6, 67, 56, 57, 22, 24, 33, 53, 61, 62, 31, 44, 35, 46, 151, 150, 40, 109, 122, 105, 113, 115, 125, 108, 81, 80], [119, 203, 202, 198, 194, 193, 26, 28, 25, 13, 66, 8, 68, 192, 197, 158, 160, 149, 42, 29, 34, 36, 45, 49, 52, 60, 10, 12, 14, 19, 54, 21, 20, 27, 32, 90, 95, 88, 208, 223, 207, 216, 224, 2, 16, 64, 18], [78, 76, 75, 79, 106, 183, 181, 1

finished, time 11.719578742980957 seconds
generation 1095, num_of_cal_pairs 4519730, best distance 567570.299370224
best route: [[68, 65, 105, 94, 136, 145, 137, 115, 106, 108, 128, 213, 206, 208, 209, 210, 215, 88, 100, 98, 86, 89, 92, 139, 138, 192, 202, 218, 121, 16, 66, 60, 59, 56, 87, 101, 197, 198, 195, 200, 214], [216, 220, 224, 217, 3, 5, 77, 84, 107, 112, 116, 83, 82, 133, 175, 168, 159, 164, 163, 172, 147, 152, 151, 170, 161, 156, 150, 53, 55, 54, 32, 38, 26, 22, 23, 44, 183, 186, 189, 42, 35, 41, 43, 39, 40, 37, 71, 81, 9, 4], [123, 211, 212, 205, 52, 51, 33, 31, 36, 34, 29, 49, 19, 2, 1, 6, 18, 64, 109, 127, 97, 95, 62, 15, 14, 113, 222, 221, 223, 119, 11, 70, 67, 130, 219, 225, 204, 199, 196, 135, 134, 80, 76, 79, 85, 126, 120, 122], [148, 144, 146, 140, 185, 177, 178, 203, 207, 201, 193, 188, 187, 179, 173, 171, 165, 160, 158, 155, 180, 184, 149, 143, 91, 47, 27, 25, 24, 28, 96, 90, 99, 103, 104, 102, 111, 114, 110, 117], [7, 13, 20, 48, 93, 142, 166, 167, 153, 45, 46, 30

In [17]:
logDir = './res/baseline/log/pr226_baseline_30_5_1000_100_10_20211216_19_57_48'
baseline_result_list = extractFinalDistListFromLogDir(logDir)


print(mannwhitneyu(final_result_list, baseline_result_list, alternative='two-sided'))
print('avg of JingranGA: {}'.format(sum(final_result_list)/30))
print('avg of baseline: {}'.format(sum(baseline_result_list)/30))

MannwhitneyuResult(statistic=0.0, pvalue=3.019859359162157e-11)
avg of JingranGA: 568002.4341119603
avg of baseline: 910403.2669450332


In [ ]:
# mtsp51 3.45s
MannwhitneyuResult(statistic=0.0, pvalue=3.019859359162157e-11)
avg of JingranGA: 580.1120456126616
avg of baseline: 918.3617979333334
# mtsp100 5.80s
MannwhitneyuResult(statistic=0.0, pvalue=3.019859359162157e-11)
avg of JingranGA: 49443.215262466234
avg of baseline: 83753.54461313334
# mtsp150 8.08s
MannwhitneyuResult(statistic=0.0, pvalue=3.019859359162157e-11)
avg of JingranGA: 94633.47115046621
avg of baseline: 143786.4989089333
# pr76 4.62s
MannwhitneyuResult(statistic=0.0, pvalue=3.019859359162157e-11)
avg of JingranGA: 204902.0892281582
avg of baseline: 338887.52780916664
# pr152 8.25s
MannwhitneyuResult(statistic=0.0, pvalue=3.019859359162157e-11)
avg of JingranGA: 328849.77738046093
avg of baseline: 541097.5042658999
# pr226 11.72s
MannwhitneyuResult(statistic=0.0, pvalue=3.019859359162157e-11)
avg of JingranGA: 568002.4341119603
avg of baseline: 910403.2669450332

In [ ]:
# mtsp51 3.45s
# mtsp100 5.80s
# mtsp150 8.08s
# pr76 4.62s
# pr152 8.25s
# pr226 11.72s